# Séléction de modèle 

In [ ]:
import matplotlib
%matplotlib nbagg
from matplotlib import pyplot

In [2]:
from statiskit import (linalg,
                       core,
                       pgm)

In [3]:
import math
import os

In [ ]:
if not 'K' in os.environ:
    os.environ['K'] = str(5)
K = int(os.environ.get('K'))
simulation = !jupyter nbconvert --ExecutePreprocessor.timeout=3600 --to notebook --execute sampling.ipynb --output sampling.ipynb
graph0 = pgm.read_gml(os.path.join('results', 'graph.gml'))
data = core.read_csv(os.path.join('results', 'data.csv'))
data

In [ ]:
if not 'LASSO' in os.environ:
    os.environ['LASSO'] = "LINEAIR"
LASSO = os.environ.get('LASSO')
lasso = !jupyter nbconvert --ExecutePreprocessor.timeout=3600 --to notebook --execute misc/LASSO.ipynb --output misc/LASSO.ipynb


In [ ]:
graphs = []
for filepath in os.listdir('.'):
    if filepath.startswith('lasso_'):
        adjmat = linalg.read_csv(filepath)
        graphs.append(pgm.UndirectedGraph(adjmat))
graphs = sorted(graphs, key = lambda graph: graph.nb_edges)
graphs

In [9]:
LLHs = []
BICs = []
AICs = []

%time
for graph in graphs:
    try:
        mle = pgm.graphical_gaussian_estimation(algo='scd',
                                                data=data,
                                                graph=graph)
        dist = mle.estimated.copy()

        LLHs.append(2 * dist.loglikelihood(data))
        
    except Exception as e:
        print e
        LLHs.append(float("nan"))
        BICs.append(LLHs[-1])
        AICs.append(LLHs[-1])
    else:
        BICs.append(LLHs[-1] - dist.nb_parameters * math.log(len(data.events)))
        AICs.append(LLHs[-1] - dist.nb_parameters * 2)

#fig = pyplot.figure()
#axes = fig.add_subplot(111)
#axes.plot(LLHs)
#axes.plot(BICs)
#axes.plot(AICs)


CPU times: user 18 µs, sys: 2 µs, total: 20 µs
Wall time: 49.8 µs


[]

In [7]:
os.environ['CRITER'] = "bic"
criter = os.environ.get('CRITER')
criter

'bic'

In [8]:
if criter == "bic":
    model = BICs.index(max(BICs))
else:
    model = AICs.index(max(AICs))
model

ValueError: max() arg is an empty sequence

In [ ]:
import math
with open(os.path.join('results', 'model.csv'), 'a') as filehandler:
    filehandler.write(','.join([str(criter), str(model), str(tp), str(tn), str(fp), str(fn)]) + '\n')